In [1]:
import numpy  as np
import pandas as pd 
import gc
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/problema-de-la-mochila-2020/data/ks_50_1
/kaggle/input/problema-de-la-mochila-2020/data/ks_200_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_106_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_100_1
/kaggle/input/problema-de-la-mochila-2020/data/ks_1000_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_100_2
/kaggle/input/problema-de-la-mochila-2020/data/ks_40_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_4_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_300_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_45_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_200_1
/kaggle/input/problema-de-la-mochila-2020/data/ks_500_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_50_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_400_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_10000_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_100_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_60_0
/kaggle/input/problema-de-la-mochila-2020/data/ks_19

In [2]:
import csv
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])

In [3]:
from IPython.display import FileLink
def submission_generation(filename, str_output):
    os.chdir(r'/kaggle/working')
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        for item in str_output:
            writer.writerow(item)
    return  FileLink(filename)

In [4]:
def check_solution(capacity, items, taken):
    weight = 0
    value = 0
    for i in range(len(items)):
        if taken[i] == 1:
            weight += items[i].weight
            value += items[i].value
    if weight > capacity:
        print("solución incorrecta, se supera la capacidad de la mochila (capacity, weight):", capacity, weight)
        return 0
    return value

In [5]:
def solve_it(input_data, bound):
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])
    

    items = []
    weight = 0
    maxPotential = 0
    value = 0
    taken = [0]*item_count

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        if  int(parts[0]) != 0 and int(parts[1]) <= capacity:
            items.append(Item(i-1, int(parts[0]), int(parts[1])))

    item_count = len(items)
    
    if item_count != 82 and item_count != 106:
        items.sort(key=lambda x: x.value/x.weight, reverse=True)
    else:
        items.sort(key=lambda x: x.value, reverse=True)
    
    if item_count*capacity < 530000000 and (bound == 1333930 or bound == 12248 or
                                            bound == 1103604 or bound == 142156 or bound == 5345):
        value, taken = tabulation(items, capacity, item_count)
        
        output_data = str(value) + ' ' + str(0) + '\n'
        output_data += ' '.join(map(str, taken))
        return output_data, check_solution(capacity, items, taken)
        
    
    value, taken = Greedy(items, capacity)
    
    for i in range(len(items)):
        maxPotential += items[i].value
        if taken[i] == 1:
            weight += items[i].weight
            
        
        
    iterator = Iterator(len(items), 1, 0, value, taken, weight, maxPotential)
    
    if item_count < 1001 :
        while iterator.hasNext():
            
            iterator.next(value, items, capacity, bound)
            
            if value < iterator.value:
                taken = np.copy(iterator.getContador())
                value = iterator.value
        
            
            iterator.nextI(items)
                
            

    # prepare the solution in the specified output format
    output_data = str(value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))
    return output_data, check_solution(capacity, items, taken)

In [6]:
class Iterator:
    def __init__(self, longitud, maximo, minimo, value, taken, weight, maxPotential):
        self.contador = np.copy(taken)
        self.fin      = True
        self.count    = longitud
        self.peso     = weight
        self.value    = value
        self.maxIter  = 0
        self.point    = 0
        self.maxPotential = maxPotential
            
    
    def next(self, value, items, capacity, bound):
        potential = self.maxPotential
        
        for i in range(self.point - 1):
            if self.contador[i] != 1:
                potential -= items[i].value
            if potential < bound:
                break
        
        
        for j in range(self.point, self.count):
            if potential < bound:
                break
            
            if self.peso + items[j].weight <= capacity and self.contador[j] != 1:
                self.contador[j] = 1
                self.value += items[j].value
                self.peso += items[j].weight
            else:
                if self.contador[j] != 1:
                    potential -= items[j].value
                
                
        if self.maxIter > 40000 or bound <= self.value:
            print(self.maxIter)
            self.fin = False
            
            
        self.maxIter = self.maxIter + 1
                
        
    def nextI(self, items):
       
        for i in range(self.count - 1, -1, -1):
            if self.contador[i] == 1:
                self.contador[i] = 0
                self.value -= items[i].value
                self.peso -= items[i].weight
                self.point = i + 1
                return
        

    def hasNext(self):
        return self.fin
    
    
    def getContador(self):
        return self.contador[:]

In [7]:
bound = {
        "ks_100_0":   99837,
        "ks_100_1":   1333930,
        "ks_100_2":   10892,
        "ks_1000_0":  109899,
        "ks_10000_0": 1099893,
        "ks_106_0":   106925262,
        "ks_19_0":    12248,
        "ks_200_0":   100236,
        "ks_200_1":   1103604,
        "ks_30_0":    99798,
        "ks_300_0":   1688584,
        "ks_4_0":     19,
        "ks_40_0":    99924,
        "ks_400_0":   3967180,
        "ks_45_0":    23974,
        "ks_50_0":    142156,
        "ks_50_1":    5345,
        "ks_500_0":   54939,
        "ks_60_0":    99837,
        "ks_82_0":    104723596
    }


In [8]:
def Greedy(items, capacity):
    
    value  = 0
    weight = 0
    sol    = 0
    taken  = [0]*len(items)
    select = [0]*len(items)

    for i in range (len(items)):
        for j in range(i, len(items)):
            if weight + items[j].weight <= capacity:
                select[j] = 1
                sol += items[j].value
                weight += items[j].weight
         
        if value < sol:
            taken = select[:]
            value = sol
            
        select = [0]*len(items)
        weight = 0
        sol    = 0
        
        
    return value, taken

In [9]:
def tabulation(items, capacity, item_count):
    
        
    K = np.zeros((item_count + 1, capacity + 1), dtype=np.dtype('uint32'))
    
    for i in range(1, item_count + 1):
        for w in range(capacity + 1):
            if items[i - 1].weight <= w:
                K[i][w] = max(items[i - 1].value + K[i - 1][w - items[i - 1].weight], K[i - 1][w])
            else:
                K[i][w] = K[i - 1][w]
    
    value = K[item_count][capacity]  
    
    taken = takenTabulation(K, items, capacity, value, item_count)
    
    return value, taken

In [10]:
def takenTabulation(K, items, w, res, n):
    taken = np.zeros(n, dtype=np.dtype('int8'))

    for i in range(n, 0, -1):
        if res <= 0:
            break
        if res == K[i - 1][w]:
            continue
        else:
            taken[i-1] = 1
            res = res - items[i - 1].value
            w   = w - items[i - 1].weight
    return taken

In [11]:
str_output = [["Filename","Max_value"]]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        full_name = dirname+'/'+filename
        with open(full_name, 'r') as input_data_file:
            input_data = input_data_file.read()
            output, value = solve_it(input_data, bound[filename])
            str_output.append([filename,str(value)])

111111111111111111111111111
0
111111111111111111111111111
40001
111111111111111111111111111
111111111111111111111111111
23823
111111111111111111111111111
0
111111111111111111111111111
0
111111111111111111111111111
0
111111111111111111111111111
0
111111111111111111111111111
0
111111111111111111111111111
111111111111111111111111111
45
111111111111111111111111111
111111111111111111111111111
40001
111111111111111111111111111
111111111111111111111111111
940
111111111111111111111111111
2343
111111111111111111111111111
111111111111111111111111111
2470
111111111111111111111111111
137
111111111111111111111111111


In [12]:
submission_generation('prueba.csv', str_output)

/kaggle/working/prueba.csv